
## FINANCIAL DATA
MODULE 5 | LESSON 3


---



# **ESTIMATING THE MARKET-IMPLIED PROBABILITY OF DEFAULT** #

|  |  |
|:---|:---|
|**Reading Time** |  25 minutes |
|**Prior Knowledge** | Probability of default, Basic Python, DataFrames  |
|**Keywords** |`.apply()`, Lambda functions, `numpy.arange()`, Symbol library |

---

*So far in this module, we have been gathering all the inputs we need for the market-implied probability of default (PD). Now that we have calculated the risk-adjusted discount rate in the last lesson, we can solve for the PD using `SymPy`. We will also refresh our understanding of some essential Python, e.g., control-flow tools, data types, as well as the append and apply methods*.

**Note:** The code that was introduced in the previous lessons is below, followed by the new code and text for this lesson.

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yfin

yfin.pdr_override()

from datetime import date
from datetime import datetime as dt
from datetime import timedelta

from sympy import solve, symbols

In [ ]:
# Loading the appropriate .csv bond data file
bond_prices_df = pd.read_csv("bond-prices.csv")

bond_prices_df

In [ ]:
def bond_dataframe_filter(df):
    # Drop bonds with missing yields and missing credit ratings
    df["Yield"].replace("", np.nan, inplace=True)
    df["Moody's®"].replace({"WR": np.nan, "": np.nan}, inplace=True)
    df["S&P"].replace({"NR": np.nan, "": np.nan}, inplace=True)
    df = df.dropna(subset=["Yield"])
    df = df.dropna(subset=["Moody's®"])
    df = df.dropna(subset=["S&P"])

    # Create Maturity Years column that aligns with Semi-Annual Payments from corporate bonds
    df["Yield"] = df["Yield"].astype(float)
    df["Coupon"] = df["Coupon"].astype(float)
    df["Price"] = df["Price"].astype(float)
    now = dt.strptime(date.today().strftime("%m/%d/%Y"), "%m/%d/%Y")
    df["Maturity"] = pd.to_datetime(df["Maturity"]).dt.strftime("%m/%d/%Y")
    daystillmaturity = []
    yearstillmaturity = []
    for maturity in df["Maturity"]:
        daystillmaturity.append((dt.strptime(maturity, "%m/%d/%Y") - now).days)
        yearstillmaturity.append((dt.strptime(maturity, "%m/%d/%Y") - now).days / 360)
    df = df.reset_index(drop=True)
    df["Maturity"] = pd.Series(daystillmaturity)
    #         `df['Maturity Years'] = pd.Series(yearstillmaturity).round()` # Better for Annual Payments
    df["Maturity Years"] = (
        round(pd.Series(yearstillmaturity) / 0.5) * 0.5
    )  # Better for Semi-Annual Payments

    # Target bonds with short-term maturities
    df["Maturity"] = df["Maturity"].astype(float)
    # `df = df.loc[df['Maturity'] >= 0]`
    years_mask = (df["Maturity Years"] > 0) & (df["Maturity Years"] <= 5)
    df = df.loc[years_mask]
    return df

In [ ]:
bond_df_result = bond_dataframe_filter(bond_prices_df)
bond_df_result

In [ ]:
# Ten-Year Risk-free Rate
timespan = 100
current_date = date.today()
past_date = current_date - timedelta(days=timespan)
ten_year_risk_free_rate_df = yfin.download("^TNX", past_date, current_date)
ten_year_risk_free_rate = (
    ten_year_risk_free_rate_df.iloc[len(ten_year_risk_free_rate_df) - 1, 4]
) / 100
ten_year_risk_free_rate

In [ ]:
# Market Risk Premium
market_risk_premium = 0.0472

In [ ]:
# Market Equity Beta
stock_market_beta = 1

In [ ]:
# Market Rate of Return
market_rate_of_return = ten_year_risk_free_rate + (
    stock_market_beta * market_risk_premium
)
market_rate_of_return

In [ ]:
# Vanguard Short-Term Corporate Bond Index Fund ETF Shares
bond_fund_ticker = "VCSH"

In [ ]:
# Download data for the bond fund and the market
market_data = yfin.download("SPY", past_date, current_date)  # the market
fund_data = yfin.download("VCSH", past_date, current_date)  # the bond fund

In [ ]:
# Approach #1 - Covariance/Variance Method:

# Calculate the covariance between the fund and the market -- this is the numerator in the Beta calculation
fund_market_cov = (
    fund_data["Adj Close"].pct_change().cov(market_data["Adj Close"].pct_change())
)
print("covariance between fund and market: ", fund_market_cov)

# Calculate market (S&P) variance -- this is the denominator in the Beta calculation
market_var = market_data["Adj Close"].pct_change().var()
print("market variance: ", market_var)

# Calculate Beta
bond_fund_beta_cv = fund_market_cov / market_var
print("bond fund beta (using covariance/variance): ", bond_fund_beta_cv)

In [ ]:
# Approach #2 - Correlation Method:

# Calculate the standard deviation of the market by taking the square root of the variance, for use in the denominator
market_stdev = market_var**0.5
print("market standard deviation: ", market_stdev)

# Calculate bond fund standard deviation, for use in the numerator

fund_stdev = fund_data["Adj Close"].pct_change().std()
print("fund standard deviation: ", fund_stdev)

# Calculate Pearson correlation between bond fund and market (S&P), for use in the numerator
fund_market_Pearson_corr = (
    fund_data["Adj Close"]
    .pct_change()
    .corr(market_data["Adj Close"].pct_change(), method="pearson")
)
print("Pearson correlation between fund and market: ", fund_market_Pearson_corr)

# Calculate Beta
fund_beta_corr = fund_stdev * fund_market_Pearson_corr / market_stdev
print("bond fund beta (using correlation): ", fund_beta_corr)

In [ ]:
# Bond's Beta: use the result of either of the two above approaches, bond_fund_beta_cv or fund_beta_corr
bond_beta = fund_beta_corr
bond_beta

In [ ]:
# Expected Risk Premium
expected_risk_premium = (market_rate_of_return - ten_year_risk_free_rate) * bond_beta
expected_risk_premium

In [ ]:
# Risk-adjusted Discount Rate
risk_adjusted_discount_rate = ten_year_risk_free_rate + expected_risk_premium
risk_adjusted_discount_rate

## 1. SymPy: Solving the Probability of Default

Given the semi-annual coupon payment frequency for the bonds we are analyzing, we can feed the annual risk-adjusted discount rate into the `bonds_probability_of_default()` function below and it will convert these annual rates into semi-annual rates.

Our last step before running the `bonds_probability_of_default()` function is to define the principal payment, the recovery rate, and the symbol for probability of default ($p$) that the `solve()` function from the Python library `SymPy` will use to calculate the probability of default by equating future expected cash flows with the current price of the corporate bond when discounted at the risk-adjusted rate.

Notice below that the numpy library method `np.append()` is different from the pandas library method `df1.append(df2)`.

In [ ]:
def bonds_probability_of_default(
    coupon, maturity_years, bond_price, principal_payment, risk_adjusted_discount_rate
):
    price = bond_price
    prob_default_exp = 0

    #     `times = np.arange(1, maturity_years+1)` # For Annual Cashflows
    #     annual_coupon = coupon # For Annual Cashflows
    times = np.arange(0.5, (maturity_years - 0.5) + 1, 0.5)  # For Semi-Annual Cashflows
    semi_annual_coupon = coupon / 2  # For Semi-Annual Cashflows

    # Calculation of Expected Cash Flow
    cashflows = np.array([])
    for i in times[:-1]:
        #         cashflows = np.append(cashflows, annual_coupon) # For Annual Cashflows
        #     cashflows = np.append(cashflows, annual_coupon+principal_payment)#  For Annual Cashflows
        cashflows = np.append(
            cashflows, semi_annual_coupon
        )  # For Semi-Annual Cashflows
    cashflows = np.append(
        cashflows, semi_annual_coupon + principal_payment
    )  # For Semi-Annual Cashflows

    for i in range(len(times)):
        #         This code block is used if there is only one payment remaining
        if len(times) == 1:
            prob_default_exp += (
                cashflows[i] * (1 - P) + cashflows[i] * recovery_rate * P
            ) / np.power((1 + risk_adjusted_discount_rate), times[i])
        #         This code block is used if there are multiple payments remaining
        else:
            #             For Annual Cashflows
            #             if times[i] == 1:
            #                 prob_default_exp += ((cashflows[i]*(1-P) + principal_payment*recovery_rate*P) / \
            #                                     np.power((1 + risk_adjusted_discount_rate), times[i]))
            #             For Semi-Annual Cashflows
            if times[i] == 0.5:
                prob_default_exp += (
                    cashflows[i] * (1 - P) + principal_payment * recovery_rate * P
                ) / np.power((1 + risk_adjusted_discount_rate), times[i])
            #             Used for either Annual or Semi-Annual Cashflows
            else:
                prob_default_exp += (
                    np.power((1 - P), times[i - 1])
                    * (cashflows[i] * (1 - P) + principal_payment * recovery_rate * P)
                ) / np.power((1 + risk_adjusted_discount_rate), times[i])

    prob_default_exp = prob_default_exp - price
    implied_prob_default = solve(prob_default_exp, P)
    implied_prob_default = round(float(implied_prob_default[0]) * 100, 2)

    if implied_prob_default < 0:
        return 0.0
    else:
        return implied_prob_default


Understand all of the code above: how the function works, how the methods work (here and in general).  Also, see the required reading section "A.7.5 Solving Equations Symbolically" and "A.7.6. Symbolic Plotting" from the following website, [which shows you how powerful and easy to use the `SymPy` library is.](https://numericalmethodssullivan.github.io/ch-python.html#solving-equations-symbolically)
<span style='color: transparent; font-size:1%'>All rights reserved WQU WorldQuant University QQQQ</span>

In [ ]:
# Variables defined for bonds_probability_of_default function
principal_payment = 100
recovery_rate = 0.40
P = symbols("P")

We are now ready to run the `bonds_probability_of_default()` function to calculate the market-implied probability of default for the chosen corporate bonds.

In [ ]:
bond_df_result.head(1)

In [ ]:
# This calculation may take some time if there are many coupon payments
bond_df_result["Probability of Default %"] = bond_df_result.head(1).apply(
    lambda row: bonds_probability_of_default(
        row["Coupon"],
        row["Maturity Years"],
        row["Price"],
        principal_payment,
        risk_adjusted_discount_rate,
    ),
    axis=1,
)

bond_df_result.head(1)

## 2. Relevant and Essential Python: Control Flow, Data Types, and Apply

[Read sections 4.7 and 4.8 of this website.](https://docs.python.org/3/tutorial/controlflow.html)

[Read section 5.6 of this website.](https://python-programming.quantecon.org/python_essentials.html#id7)

[Read this website.](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html)

These resources will help you understand the dense line of code above.

## 3. Conclusion

In this lesson, we used the inputs we found in the previous lessons to finally estimate the probability of default for the bond issuer of our choice. We had to provide an assumed recovery rate, and then use a solver (the `solve()` method) embedded in the `bonds_probability_of_default()` function, to find the probability of default implied by the price of the relevant bonds.

In the next lesson, we look at the ratings for these bonds provided by the ratings agencies. We will download the transition matrices (which include the probability of default for each rating) and compare the probability of default (PD) that we calculated in this lesson to the PD associated with the ratings agencies' ratings.


**References**

* "More Control Flow Tools." Python.org.  https://docs.python.org/3/tutorial/controlflow.html

* Sargent, Thomas J., and John Stachurski. "Python Programming for Finance and Economics." QuantEcon.org, https://python-programming.quantecon.org/python_essentials.html#id7

* "Pandas.DataFrame.apply." Pandas.pydata.org. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html

* The  code and related documentation used in this lesson is adapted from: <br>**Hugh Donnelly, CFA**<br>*AlphaWave Data* <br> **March 2021** under the following  MIT License:

> Copyright (c) 2020 HDVI <br>Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:<br>The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.<br>THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

**Note:** The above MIT license notice is copied here to comply with its requirements but it does **not** apply to the content in these lesson notes.


---
Copyright 2023 WorldQuant University. This
content is licensed solely for personal use. Redistribution or
publication of this material is strictly prohibited.
